In [1]:
import sys
from pathlib import Path

# Prior to running this notebook,
# I ran pixi run install-editable to use the edited version of GSAS-II
from GSASII import GSASIIscriptable as G2

GSAS-II binaries co-located with GSAS-II: /nsls2/users/acorrao/GitHub/GSAS-II/GSASII
6 values read from /nsls2/users/acorrao/.GSASII/config.ini


# Init proj, add hist, add phase

In [2]:
# 1. Input file paths (edit these to actual file locations)
assets_path = Path("/nsls2/users/acorrao/GitHub/GSAS-II/test-G2sc-size-strain")  # path to assets (data, CIF, instrument files)

gpxname = assets_path / "LaB6_660c_Rietveld.gpx"
data_file = assets_path / 'xrd_LaB6_660c_std_brac2_20250724-194924_70c707_primary-1_mean_tth.chi' # Diffraction data file for LaB6
instrument_file = assets_path / "PDF_1m.instprm" # Instrument parameter file (contains initial U,V,W,X,Y, wavelength, etc.)
cif_file = assets_path / "LaB6_SRM_660c.CIF" # CIF file for LaB6 (with known lattice parameter)


# 2. Create a new GSAS-II project, add hist, add phase
proj = G2.G2Project(newgpx=str(gpxname))
hist = proj.add_powder_histogram(data_file, instrument_file)
phase = proj.add_phase(cif_file, phasename="LaB6", histograms=[hist], fmthint="CIF")

/nsls2/users/acorrao/GitHub/GSAS-II/test-G2sc-size-strain/xrd_LaB6_660c_std_brac2_20250724-194924_70c707_primary-1_mean_tth.chi read by Reader Topas xye/qye or 2th Fit2D chi/qchi
Instrument parameters read: /nsls2/users/acorrao/GitHub/GSAS-II/test-G2sc-size-strain/PDF_1m.instprm (G2 fmt) bank None
/nsls2/users/acorrao/GitHub/GSAS-II/test-G2sc-size-strain/LaB6_SRM_660c.CIF read by Reader CIF
gpx file saved as /nsls2/users/acorrao/GitHub/GSAS-II/test-G2sc-size-strain/LaB6_660c_Rietveld.gpx


In [3]:
# 3. Print initial values of size and strain parameters
########################
# Multi-phase approach if phase objects not returned to a variable:
# phase_names = [p.name for p in proj.phases()]

# for phase_name in phase_names:
#     print("Initial Size parameters:\n",proj.data['Phases'][phase_name]['Histograms'][hist.name]['Size'])
#     print("Initial Strain parameters:\n",proj.data['Phases'][phase_name]['Histograms'][hist.name]['Mustrain'])
########################

# Alternative phase object access:
print("Initial Size parameters:\n",phase.data['Histograms'][hist.name]['Size'])
print("Initial Strain parameters:\n",phase.data['Histograms'][hist.name]['Mustrain'])


Initial Size parameters:
 ['isotropic', [1.0, 1.0, 1.0], [False, False, False], [0, 0, 1], [1.0, 1.0, 1.0, 0.0, 0.0, 0.0], [False, False, False, False, False, False]]
Initial Strain parameters:
 ['isotropic', [1000.0, 1000.0, 1.0], [False, False, False], [0, 0, 1], [0.01, 0.01], [False, False]]


In [4]:
# 4. Define phase-specific size-strain parameters
size_strain_dict = {
    "Size": {"Type": "isotropic", "Value": 0.1, "refine": True, "LGmix": {"value": 0.5, "refine": True}},
    "Mustrain": {"Type": "isotropic", "Value": 0.01, "refine": True, "LGmix": {"value": 0.5, "refine": True}}
}

phase.set_HAP_refinements(size_strain_dict)

# 5. Print updated values of size and strain parameters to verify they are set correctly
print("Updated Size parameters:\n",phase.data['Histograms'][hist.name]['Size'])
print("Updated Strain parameters:\n",phase.data['Histograms'][hist.name]['Mustrain'])

Updated Size parameters:
 ['isotropic', [1.0, 1.0, 0.5], [True, False, True], [0, 0, 1], [1.0, 1.0, 1.0, 0.0, 0.0, 0.0], [False, False, False, False, False, False]]
Updated Strain parameters:
 ['isotropic', [1000.0, 1000.0, 0.5], [True, False, True], [0, 0, 1], [0.01, 0.01], [False, False]]


In [5]:
# 6. Define refinement settings for the project

refine_dict = {
    "set": {
        "Cell": False,
        "Sample Parameters": ["Scale"],
        "Background": {"type": "chebyschev-1", "no. coeffs": 4, "refine": True}  # 4-term Chebyshev background
    }
}

# 7. Apply refinement settings to the project
proj.set_refinement(refine_dict)

In [6]:
# 8. Run the refinement
proj.refine()

# 9. Print Rwp, size, and strain values after refinement
print("\n\nFinal Rwp:", hist.residuals.get("wR", "No Rwp value found - check residuals keys"))
print("Final Size parameters:\n",phase.data['Histograms'][hist.name]['Size'])
print("Final Strain parameters:\n",phase.data['Histograms'][hist.name]['Mustrain'])

gpx file saved as /nsls2/users/acorrao/GitHub/GSAS-II/test-G2sc-size-strain/LaB6_660c_Rietveld.gpx
 Hessian Levenberg-Marquardt SVD refinement on 9 variables:
initial chi^2 7.3278e+07 with 4096 obs.
Cycle 0: 0.11s, Chi**2: 1985 for 4096 obs., Lambda: 0,  Delta: 1, SVD=0
divergence: chi^2 21466 on 4096 obs. (0 SVD zeros)
	increasing Marquardt lambda to 1.0e-03
divergence: chi^2 9962 on 4096 obs. (0 SVD zeros)
	increasing Marquardt lambda to 1.0e-02
divergence: chi^2 2326.4 on 4096 obs. (0 SVD zeros)
	increasing Marquardt lambda to 1.0e-01
divergence: chi^2 7194.9 on 4096 obs. (0 SVD zeros)
	increasing Marquardt lambda to 1.0e+00
divergence: chi^2 3007.3 on 4096 obs. (0 SVD zeros)
	increasing Marquardt lambda to 1.0e+01
Cycle 1: 2.10s, Chi**2: 1713.5 for 4096 obs., Lambda: 10,  Delta: 0.137, SVD=0
Cycle 2: 0.11s, Chi**2: 1422.4 for 4096 obs., Lambda: 10,  Delta: 0.17, SVD=0
Maximum shift/esd = 422.242 for all cycles
Note highly correlated parameters:
 ** 0:0:Size;mx and 0:0:Size;i (@99.1

# 